In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import cufflinks as cf
%matplotlib inline
from nltk import DecisionTreeClassifier, accuracy, metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import  RandomizedSearchCV, train_test_split
#from xgboost import XGBClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.impute import  SimpleImputer
#"reade csv file "
data = pd.read_csv("D:\Documents\Ganna allah khaled\courses\data mining\heart.csv")
print(data.head())
print(data.shape)

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   3       145   233    1        0      150      0      2.3      0   
1   37    1   2       130   250    0        1      187      0      3.5      0   
2   41    0   1       130   204    0        0      172      0      1.4      2   
3   56    1   1       120   236    0        1      178      0      0.8      2   
4   57    0   0       120   354    0        1      163      1      0.6      2   

   ca  thal  target  
0   0     1       1  
1   0     2       1  
2   0     2       1  
3   0     2       1  
4   0     2       1  
(303, 14)


In [38]:
data.drop_duplicates(keep = 'first', inplace = True)#remove duplicate rows
print(data)

#remove values not in the range of features
data.loc[data['ca']==4,'ca']=np.NAN
data.loc[data['thal'] ==0,'thal']=np.NAN


#data=data.dropna(axis=0)

data = data.fillna(data.median())
#data.isnull().sum()
print(data)
print("##############################################")
#figure that describe the outliers of features


#remove outliers
for x in data:
    q75,q25 = np.percentile(data.loc[:,x],[75,25])
    intr_qr = q75-q25

    max = q75+(1.5*intr_qr)
    min = q25-(1.5*intr_qr)

    data.loc[data[x] < min,x] = np.nan
    data.loc[data[x] > max,x] = np.nan
print(data.isnull().sum())

#data = data.dropna(axis = 0)
data = data.fillna(data.median())
#data.isnull().sum()
print(data.isnull().sum())

print("#########################################################")
print(data)

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  target  
0        0   0     1       1  
1        0   0     2     

In [40]:
from sklearn.model_selection import train_test_split

predictors = data.drop("target",axis=1)
target = data["target"]

X_train,X_test,Y_train,Y_test = train_test_split(predictors,target,test_size=0.20,random_state=0)


knn = KNeighborsClassifier(n_neighbors = 8)
knn.fit(X_train,Y_train)
y_pred1 = knn.predict(X_test)
score_nb = round(accuracy_score(y_pred1,Y_test)*100,2)

print("The accuracy score achieved using KNN is: "+str(score_nb)+" %")

from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb.fit(X_train,Y_train)

Y_pred_nb = nb.predict(X_test)
score_nb = round(accuracy_score(Y_pred_nb,Y_test)*100,2)

print("The accuracy score achieved using Naive Bayes is: "+str(score_nb)+" %")

from sklearn.tree import DecisionTreeClassifier

max_accuracy = 0

for x in range(302):
    dt = DecisionTreeClassifier(random_state=x)
    dt.fit(X_train, Y_train)
    Y_pred_dt = dt.predict(X_test)
    current_accuracy = round(accuracy_score(Y_pred_dt, Y_test) * 100, 2)
    if (current_accuracy > max_accuracy):
        max_accuracy = current_accuracy
        best_x = x




dt = DecisionTreeClassifier(random_state=best_x)
dt.fit(X_train, Y_train)
Y_pred_dt = dt.predict(X_test)
score_dt = round(accuracy_score(Y_pred_dt,Y_test)*100,2)
clf_report = pd.DataFrame(classification_report(Y_test, Y_pred_dt, output_dict=True))
print("The accuracy score achieved using Decision Tree is: "+str(score_dt)+" %")
print(clf_report)

from sklearn.ensemble import RandomForestClassifier

max_accuracy = 0

for x in range(302):
    rf = RandomForestClassifier(random_state=x)
    rf.fit(X_train, Y_train)
    Y_pred_rf = rf.predict(X_test)
    current_accuracy = round(accuracy_score(Y_pred_rf, Y_test) * 100, 2)
    if (current_accuracy > max_accuracy):
        max_accuracy = current_accuracy
        best_x = x



rf = RandomForestClassifier(random_state=best_x)
rf.fit(X_train, Y_train)
Y_pred_rf = rf.predict(X_test)

score_rf = round(accuracy_score(Y_pred_rf,Y_test)*100,2)

print("The accuracy score achieved using random classifier is: "+str(score_rf)+" %")

The accuracy score achieved using KNN is: 67.21 %
The accuracy score achieved using Naive Bayes is: 88.52 %
The accuracy score achieved using Decision Tree is: 90.16 %
                 0.0        1.0  accuracy  macro avg  weighted avg
precision   0.862069   0.937500  0.901639   0.899784      0.904112
recall      0.925926   0.882353  0.901639   0.904139      0.901639
f1-score    0.892857   0.909091  0.901639   0.900974      0.901905
support    27.000000  34.000000  0.901639  61.000000     61.000000
The accuracy score achieved using random classifier is: 91.8 %


In [ ]:
plt.figure(figsize=(20,12))
sns.set_context('notebook',font_scale = 1.3)
sns.heatmap(data.corr(),annot=True,linewidth =2)
plt.tight_layout()
plt.show()

In [ ]:
sns.set_context('notebook',font_scale = 2.3)
data.drop('target', axis=1).corrwith(data.target).plot(kind='bar', grid=True, figsize=(20, 10),
title="Correlation with the target feature")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(25,12))
sns.set_context('notebook',font_scale = 1.5)
sns.barplot(x=data.age.value_counts()[:10].index,y=data.age.value_counts()[:10].values)
plt.tight_layout()
"from figure we can see that the 58 age column has the highest frequency."

In [ ]:
" divide the Age feature into three parts  “Young”, “Middle” and “Elder”"
Young = data[(data.age>=29)&(data.age<40)]
Middle = data[(data.age>=40)&(data.age<55)]
Elder = data[(data.age>55)]

plt.figure(figsize=(23,10))
sns.set_context('notebook',font_scale = 1.5)
sns.barplot(x=['young ages','middle ages','elderly ages'],y=[len(Young),len(Middle),len(Elder)])
plt.tight_layout()

"we can see that elder people are the most affected by heart disease and young ones are the least affected."

In [ ]:
colors = ['blue','green','yellow']
explode = [0,0,0.1]
plt.figure(figsize=(10,10))
sns.set_context('notebook',font_scale = 1.2)
plt.pie([len(Young),len(Middle),len(Elder)],labels=['young ages','middle ages','elderly ages'],explode=explode,colors=colors, autopct='%1.1f%%')
plt.tight_layout()
"to prove above inference"

In [ ]:
plt.figure(figsize=(18,9))
sns.set_context('notebook',font_scale = 1.5)
sns.countplot(data['sex'])
plt.tight_layout()

In [ ]:
"let’s plot the relation between sex and slope."
plt.figure(figsize=(18,9))
sns.set_context('notebook',font_scale = 1.5)
sns.countplot(data['sex'],hue=data["slope"])
plt.tight_layout()
"Inference: Here it is clearly visible that the slope value is higher in the case of males(1)."

In [ ]:
"Chest Pain Type(“cp”) Analysis"
plt.figure(figsize=(18,9))
sns.set_context('notebook',font_scale = 1.5)
sns.countplot(data['cp'])
plt.tight_layout()
"Inference: As seen, there are 4 types of chest pain"

In [ ]:
"Analyzing cp vs target column"
plt.figure(figsize=(18,9))
sns.set_context('notebook',font_scale = 1.5)
sns.countplot(data['cp'],hue=data["slope"])
plt.tight_layout()
"Inference: From the above graph we can make some inferences",

"People having the least chest pain are not likely to have heart disease."
"People having severe chest pain are likely to have heart disease."
"Elderly people are more likely to have chest pain."

In [ ]:
"Thal Analysis"
plt.figure(figsize=(18,9))
sns.set_context('notebook',font_scale = 1.5)
sns.countplot(data['thal'])
plt.tight_layout()

In [ ]:
data.target.value_counts()

In [ ]:
plt.figure(figsize=(18,9))
sns.set_context('notebook',font_scale = 1.5)
sns.countplot(data['target'])
plt.tight_layout()